2018 Census Data for Selected Variables - Baltimore City

In [1]:
# You may need to install some of the dependencies below using "pip install"
# pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
# conda install -c conda-forge cenpy

In [4]:
# conda update -n base -c defaults conda


In [5]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.
# pip install cenpy

In [6]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2018)

# Configure gmaps
#gmaps.configure(api_key=gkey)

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\libpysal\weights\util.py:23: UserWarning: geopandas not available. Some functionality will be disabled.
  warn("geopandas not available. Some functionality will be disabled.")


In [8]:
# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2018')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2018_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you, returned as a pandas DataFrame.
print(f"ACSDT5Y2018 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2018 provides 27037 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total!!Female!!Service occupations!!...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE P...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 201...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN


In [9]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# I'll use "Nicar20" as citation for the above site from here on out
# Values for 'group' are ACS table IDs; 
# For this data, when it's N/A, it's for other kinds of API variables so won't include those:
short_vars = variables[~(variables['group'] == 'N/A')] 

# Get a list of table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(10) 

,group,concept
B00001_001E,B00001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION
B00002_001E,B00002,UNWEIGHTED SAMPLE HOUSING UNITS
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ...
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ...
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)


In [10]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# "attributes" column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
# Just fyi, The other two which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total!!Male,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total!!Male!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total!!Male!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total!!Male!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total!!Male!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total!!Male!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total!!Male!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total!!Male!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total!!Male!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [11]:
# search function
# sample = censusdata.search('acs5', 2018, 'concept', 'age ')
# print(sample)

#  to see list of all tables in the ACS5:  c.acs5.tables()

In [12]:
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B08604'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08604_001E  | WORKER POPULATION FOR WORKPLAC | !! Estimate Total                                        | int  
-------------------------------------------------------------------------------------------------------------------


In [13]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510; * is to pull data for all census tracts in the 510 FIPs
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_005E",              
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",                                    
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,37.0,424.0,108.0,87.0,...,27.0,13.0,5.0,609.0,874.0,259.0,309.0,24,510,130805
1,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,206.0,1442.0,275.0,435.0,...,70.0,23.0,38.0,787.0,2097.0,398.0,569.0,24,510,210100
2,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,169.0,1539.0,560.0,115.0,...,0.0,20.0,30.0,0.0,2045.0,0.0,1139.0,24,510,270701
3,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,121.0,611.0,413.0,25.0,...,34.0,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,190100
4,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,117.0,918.0,179.0,155.0,...,81.0,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,190200


In [14]:
# Pull values from the ACS 5 yr. census codes/ variables listed and save them in the variable "age_data"

age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,17.0,71.0,40.0,34.0,18.0,30.0,32.0,24,510,130805
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,26.0,10.0,14.0,54.0,5.0,22.0,0.0,24,510,210100
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,47.0,10.0,13.0,7.0,35.0,16.0,5.0,24,510,270701
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,8.0,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,6.0,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200


In [15]:
# Sum columns of age groups that are under 18 years old for male and female and add new column "Pop. <18 years"
columns_under18 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_006E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E",
                   "B01001_030E"]
age_pd['Pop. <18 years']= age_pd[columns_under18].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,71.0,40.0,34.0,18.0,30.0,32.0,24,510,130805,83.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,10.0,14.0,54.0,5.0,22.0,0.0,24,510,210100,511.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,10.0,13.0,7.0,35.0,16.0,5.0,24,510,270701,460.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100,650.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200,446.0


In [16]:
# Sum columns of age groups that are 18-64 years old (working age) for male and female and add new column "Pop. working age"
columns_working_age = [                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E"]    
                           
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()         
            
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,40.0,34.0,18.0,30.0,32.0,24,510,130805,83.0,486.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,14.0,54.0,5.0,22.0,0.0,24,510,210100,511.0,1862.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,13.0,7.0,35.0,16.0,5.0,24,510,270701,460.0,1882.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,7.0,25.0,53.0,38.0,27.0,24,510,190100,650.0,1201.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,32.0,12.0,26.0,0.0,0.0,24,510,190200,446.0,1293.0


In [17]:
# Sum columns of age groups that are 65+ years old for male and female and add new column "Pop. 65+ years"
columns_senior = ["B01001_020E",
                  "B01001_021E",
                  "B01001_022E",
                  "B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_044E",
                  "B01001_045E",
                  "B01001_046E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. 65+ years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,34.0,18.0,30.0,32.0,24,510,130805,83.0,486.0,374.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,54.0,5.0,22.0,0.0,24,510,210100,511.0,1862.0,159.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,7.0,35.0,16.0,5.0,24,510,270701,460.0,1882.0,126.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,25.0,53.0,38.0,27.0,24,510,190100,650.0,1201.0,280.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,12.0,26.0,0.0,0.0,24,510,190200,446.0,1293.0,147.0


In [18]:
age_final = age_pd[[ "tract", "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]
age_final.head()

,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,130805,83.0,486.0,374.0
1,210100,511.0,1862.0,159.0
2,270701,460.0,1882.0,126.0
3,190100,650.0,1201.0,280.0
4,190200,446.0,1293.0,147.0


In [19]:
age_final = age_final.rename(columns={"tract": "Census_tract"})
age_final

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,130805,83.0,486.0,374.0
1,210100,511.0,1862.0,159.0
2,270701,460.0,1882.0,126.0
3,190100,650.0,1201.0,280.0
4,190200,446.0,1293.0,147.0
...,...,...,...,...
195,110200,157.0,4647.0,902.0
196,020300,243.0,3317.0,317.0
197,150400,539.0,2851.0,593.0
198,010200,340.0,2300.0,229.0


In [20]:
# Did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",382638.0,24,510


In [21]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,37.0,424.0,108.0,87.0,...,13.0,5.0,609.0,874.0,259.0,309.0,24,510,130805,24510130805
1,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,206.0,1442.0,275.0,435.0,...,23.0,38.0,787.0,2097.0,398.0,569.0,24,510,210100,24510210100
2,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,169.0,1539.0,560.0,115.0,...,20.0,30.0,0.0,2045.0,0.0,1139.0,24,510,270701,24510270701
3,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,121.0,611.0,413.0,25.0,...,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,190100,24510190100
4,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,117.0,918.0,179.0,155.0,...,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,190200,24510190200


In [22]:
# "Number of rows" = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 23)


In [23]:
# remove extraneous column "tract"
census_pd = census_pd.drop(["tract"], axis=1)
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,GEOID
0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,37.0,424.0,108.0,87.0,...,27.0,13.0,5.0,609.0,874.0,259.0,309.0,24,510,24510130805
1,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,206.0,1442.0,275.0,435.0,...,70.0,23.0,38.0,787.0,2097.0,398.0,569.0,24,510,24510210100
2,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,169.0,1539.0,560.0,115.0,...,0.0,20.0,30.0,0.0,2045.0,0.0,1139.0,24,510,24510270701
3,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,121.0,611.0,413.0,25.0,...,34.0,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,24510190100
4,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,117.0,918.0,179.0,155.0,...,81.0,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,24510190200


In [24]:
# GIS Analyst, Patrick, provided a csv (created from ARcGIS) that contains a key to match Baltimore's commercial corridors 
# with specific GEOIDS 

# Store filepath in a variable
corridor_key = "./CSVs/corr_key.csv"

# Read the file with the pandas library
corr_key_df = pd.read_csv(corridor_key)
corr_key_df.dtypes


GEOID        int64
Corridor    object
dtype: object

In [25]:
corr_key_df

,GEOID,Corridor
0,24510260403,Highlandtown
1,24510230200,Hamilton Lauraville
2,24510260102,Pimlico
3,24510260303,Hamilton Lauraville
4,24510260800,Highlandtown
...,...,...
195,24510220100,NaN
196,24510230300,NaN
197,24510250207,NaN
198,24510250303,NaN


In [26]:
# combine age data with rest of census data
census_joined = pd.concat([age_final, census_pd], axis="columns")


census_joined.shape

(200, 26)

In [27]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_joined = census_joined.rename(columns={"B01003_001E": "Population",
                                      "tract": "Census Tract",        
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add a new column for poverty rate (Poverty Count / Population)
census_joined["Poverty rate"] = 100 * \
    census_joined["Poverty count"].astype(
        int) / census_joined["Population"].astype(int)

# Add a new column for unemployment rate (Employment Count / Population)
census_joined["Unemployment rate"] = 100 * \
    census_joined["Unemployment count"].astype(
        int) / census_joined["Population"].astype(int)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate
0,130805,83.0,486.0,374.0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,...,5.0,609.0,874.0,259.0,309.0,24,510,24510130805,18.133616,3.923648
1,210100,511.0,1862.0,159.0,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,...,38.0,787.0,2097.0,398.0,569.0,24,510,24510210100,17.101106,8.135861
2,270701,460.0,1882.0,126.0,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,...,30.0,0.0,2045.0,0.0,1139.0,24,510,24510270701,18.679092,6.847650
3,190100,650.0,1201.0,280.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,62.0,508.0,1506.0,156.0,652.0,24,510,24510190100,50.070389,5.678085
4,190200,446.0,1293.0,147.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,233.0,485.0,1481.0,200.0,498.0,24,510,24510190200,31.336161,6.203606


In [28]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_joined['Total owner-occupied units'] + census_joined['Total renter-occupied units']
census_joined["Total occupied units"] = sum_column

In [29]:
census_joined["Home ownership rate"] = 100 * \
    census_joined["Total owner-occupied units"].astype(
        int) / census_joined["Total occupied units"].astype(
        int) 

census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,130805,83.0,486.0,374.0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,...,874.0,259.0,309.0,24,510,24510130805,18.133616,3.923648,568.0,45.598592
1,210100,511.0,1862.0,159.0,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,...,2097.0,398.0,569.0,24,510,24510210100,17.101106,8.135861,967.0,41.158221
2,270701,460.0,1882.0,126.0,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,...,2045.0,0.0,1139.0,24,510,24510270701,18.679092,6.847650,1139.0,0.000000
3,190100,650.0,1201.0,280.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,1506.0,156.0,652.0,24,510,24510190100,50.070389,5.678085,808.0,19.306931
4,190200,446.0,1293.0,147.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,1481.0,200.0,498.0,24,510,24510190200,31.336161,6.203606,698.0,28.653295


In [30]:
# Round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
census_joined["Home ownership rate"] = census_joined["Home ownership rate"].astype(float).round(1)


In [31]:
census_joined["Poverty rate"] = census_joined["Poverty rate"].astype(float).round(1)


In [32]:
census_joined["Unemployment rate"] = census_joined["Unemployment rate"].astype(float).round(1)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,B28007_001E,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,130805,83.0,486.0,374.0,"Census Tract 1308.05, Baltimore city, Maryland",39750.0,943.0,57.9,33640.0,171.0,...,874.0,259.0,309.0,24,510,24510130805,18.1,3.9,568.0,45.6
1,210100,511.0,1862.0,159.0,"Census Tract 2101, Baltimore city, Maryland",63558.0,2532.0,31.1,30254.0,433.0,...,2097.0,398.0,569.0,24,510,24510210100,17.1,8.1,967.0,41.2
2,270701,460.0,1882.0,126.0,"Census Tract 2707.01, Baltimore city, Maryland",36994.0,2468.0,31.9,28810.0,461.0,...,2045.0,0.0,1139.0,24,510,24510270701,18.7,6.8,1139.0,0.0
3,190100,650.0,1201.0,280.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,1506.0,156.0,652.0,24,510,24510190100,50.1,5.7,808.0,19.3
4,190200,446.0,1293.0,147.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,1481.0,200.0,498.0,24,510,24510190200,31.3,6.2,698.0,28.7


In [33]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# Density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [34]:
census_joined.count()

Census_tract                                      200
Pop. <18 years                                    200
Pop. working age                                  200
Pop. 65+ years                                    200
Name                                              200
Median household income                           200
Population                                        200
Median age                                        200
Per capita income                                 200
Poverty count                                     200
Unemployment count                                200
# employed, age 16+                               200
# persons age 25+ graduated high school           200
# persons age 25+ with Bachelor's degree          200
Pop. white                                        200
Pop. Black                                        200
Pop. Asian                                        200
Pop. 2 or more races                              200
Pop. Hispanic origin        

In [35]:
census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Name                                               object
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more

In [36]:
# Remove "Name" column because it's extraneous
census_joined = census_joined.drop(["Name"], axis=1)

census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 

In [37]:
# Split the "Name" column into 3 separate columns: "Census_Tract", "County", "State"
# census_joined[['Census_Tract', "County", "State"]]= census_joined['Name'].str.split(",", n=3, expand=True)
# census_joined.head()

In [38]:
# List the columns in the census_joined dataframe
census_joined.columns

Index(['Census_tract', 'Pop. <18 years', 'Pop. working age', 'Pop. 65+ years',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Unemployment count',
       '# employed, age 16+', '# persons age 25+ graduated high school',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. 2 or more races', 'Pop. Hispanic origin',
       'Total pop. in occupied housing units by tenure', 'B28007_001E',
       'Total owner-occupied units', 'Total renter-occupied units', 'State',
       'county', 'GEOID', 'Poverty rate', 'Unemployment rate',
       'Total occupied units', 'Home ownership rate'],
      dtype='object')

In [39]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_joined["Census_tract"].unique())
tract_count

200

In [40]:
# make sure that the GEOID is the same data type in each of the dataframes to be merged, by using .astype
#(int64 is a 64-bit integer, refers to how much storage needed for this datapoint)
census_joined["GEOID"] = census_joined["GEOID"].astype('int64')


In [41]:
census_joined.dtypes

Census_tract                                       object
Pop. <18 years                                    float64
Pop. working age                                  float64
Pop. 65+ years                                    float64
Median household income                           float64
Population                                        float64
Median age                                        float64
Per capita income                                 float64
Poverty count                                     float64
Unemployment count                                float64
# employed, age 16+                               float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. Asian                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 

In [42]:
corr_key_df["GEOID"] = corr_key_df["GEOID"].astype('int64')

In [43]:
corr_key_df.dtypes

GEOID        int64
Corridor    object
dtype: object

In [44]:
# merge the census_joined dataframe with the corr_key_df dataframe on the common column "GEOID"

corridors_df = pd.merge(
    census_joined, corr_key_df, on="GEOID")

# remove any columns with NaN ("Not a Number") - used for missing values, by using .dropna()
corridors_df = corridors_df.dropna()


In [45]:
# reset the index numbers for the dataframe (the first column)
corridors_df = corridors_df.reset_index(drop=True)
corridors_df.head()
# remove the "county" column as it is not needed
#corridors_df = corridors_df.drop(["county"], axis=1)


,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,...,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate,Corridor
0,210100,511.0,1862.0,159.0,63558.0,2532.0,31.1,30254.0,433.0,206.0,...,398.0,569.0,24,510,24510210100,17.1,8.1,967.0,41.2,Pigtown
1,120202,159.0,5626.0,190.0,36435.0,5975.0,21.5,16907.0,1078.0,173.0,...,313.0,1325.0,24,510,24510120202,18.0,2.9,1638.0,19.1,Waverly
2,150100,518.0,1291.0,383.0,17346.0,2192.0,40.2,17457.0,1113.0,150.0,...,343.0,605.0,24,510,24510150100,50.8,6.8,948.0,36.2,Penn Ave
3,200800,551.0,1328.0,249.0,40789.0,2128.0,37.9,19553.0,564.0,85.0,...,448.0,342.0,24,510,24510200800,26.5,4.0,790.0,56.7,Irvington
4,260301,1713.0,2847.0,320.0,43951.0,4880.0,27.4,16972.0,1630.0,273.0,...,928.0,645.0,24,510,24510260301,33.4,5.6,1573.0,59.0,Belair Rd


In [46]:
# Change order of columns in DataFrame by using double brackets
# Notice that columns such as "state" and "county" were not included in the code below and they are no longer included in the dataframe (they're extraneous, we don't need them)
corridors_df = corridors_df[["Corridor", "Census_tract", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Pop. Asian","Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]

corridors_df.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Pigtown,210100,24510210100,2532.0,63558.0,30254.0,433.0,17.1,8.1,1442.0,...,1683.0,23.0,38.0,70.0,787.0,398.0,569.0,511.0,1862.0,159.0
1,Waverly,120202,24510120202,5975.0,36435.0,16907.0,1078.0,18.0,2.9,2271.0,...,633.0,402.0,444.0,1485.0,673.0,313.0,1325.0,159.0,5626.0,190.0
2,Penn Ave,150100,24510150100,2192.0,17346.0,17457.0,1113.0,50.8,6.8,631.0,...,2103.0,34.0,0.0,26.0,1084.0,343.0,605.0,518.0,1291.0,383.0
3,Irvington,200800,24510200800,2128.0,40789.0,19553.0,564.0,26.5,4.0,794.0,...,1841.0,49.0,70.0,0.0,1210.0,448.0,342.0,551.0,1328.0,249.0
4,Belair Rd,260301,24510260301,4880.0,43951.0,16972.0,1630.0,33.4,5.6,1816.0,...,4407.0,160.0,10.0,0.0,2360.0,928.0,645.0,1713.0,2847.0,320.0


In [47]:
# sort the column "Census_tract" so that it is easier to compare with other years that we are going to pull in another notebook
census_2018_FINAL =corridors_df.sort_values("Census_tract")
census_2018_FINAL = census_2018_FINAL.reset_index(drop=True)
census_2018_FINAL

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,2869.0,107617.0,56058.0,169.0,5.9,1.0,2100.0,...,107.0,53.0,69.0,107.0,2332.0,1103.0,290.0,340.0,2300.0,229.0
1,Brooklyn,020300,24510020300,3877.0,108516.0,87413.0,359.0,9.3,1.4,2912.0,...,89.0,229.0,265.0,364.0,1687.0,793.0,1350.0,243.0,3317.0,317.0
2,E Monument St,040100,24510040100,4404.0,62500.0,49876.0,870.0,19.8,1.5,2863.0,...,894.0,243.0,380.0,913.0,270.0,193.0,2403.0,412.0,3887.0,105.0
3,Hamilton Lauraville,040200,24510040200,901.0,34375.0,16596.0,66.0,7.3,1.9,367.0,...,311.0,55.0,26.0,123.0,0.0,0.0,228.0,18.0,872.0,11.0
4,E Monument St,060200,24510060200,3241.0,72465.0,32023.0,864.0,26.7,4.5,1681.0,...,1370.0,96.0,430.0,58.0,1574.0,534.0,545.0,836.0,2226.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Waverly,280302,24510280302,2464.0,43438.0,25487.0,438.0,17.8,4.5,1207.0,...,2193.0,168.0,14.0,7.0,869.0,309.0,690.0,619.0,1578.0,267.0
121,Pimlico,280401,24510280401,3479.0,46139.0,28736.0,456.0,13.1,3.1,1608.0,...,2722.0,132.0,62.0,16.0,2222.0,923.0,596.0,676.0,2220.0,583.0
122,Pigtown,280402,24510280402,1634.0,48276.0,22145.0,219.0,13.4,6.9,752.0,...,1567.0,1.0,24.0,0.0,1111.0,446.0,158.0,256.0,1057.0,321.0
123,Belair Rd,280403,24510280403,5264.0,62945.0,37610.0,294.0,5.6,3.9,2801.0,...,3848.0,56.0,136.0,37.0,2310.0,881.0,1259.0,1199.0,3409.0,656.0


In [48]:
census_2018_formatted = census_2018_FINAL
census_2018_formatted.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,2869.0,107617.0,56058.0,169.0,5.9,1.0,2100.0,...,107.0,53.0,69.0,107.0,2332.0,1103.0,290.0,340.0,2300.0,229.0
1,Brooklyn,020300,24510020300,3877.0,108516.0,87413.0,359.0,9.3,1.4,2912.0,...,89.0,229.0,265.0,364.0,1687.0,793.0,1350.0,243.0,3317.0,317.0
2,E Monument St,040100,24510040100,4404.0,62500.0,49876.0,870.0,19.8,1.5,2863.0,...,894.0,243.0,380.0,913.0,270.0,193.0,2403.0,412.0,3887.0,105.0
3,Hamilton Lauraville,040200,24510040200,901.0,34375.0,16596.0,66.0,7.3,1.9,367.0,...,311.0,55.0,26.0,123.0,0.0,0.0,228.0,18.0,872.0,11.0
4,E Monument St,060200,24510060200,3241.0,72465.0,32023.0,864.0,26.7,4.5,1681.0,...,1370.0,96.0,430.0,58.0,1574.0,534.0,545.0,836.0,2226.0,179.0


In [49]:
# Use .map to format columns (helpful resource for this: https://towardsdatascience.com/apply-thousand-separator-and-other-formatting-to-pandas-dataframe-45f2f4c7ab01)
# Note: once you format values in a column, they are changed to strings (see cell below to see data types of each column)
# I will use the census_2017_FINAL dataframe to use for analysis as needed (can do calculations with number data types but not strings)
# You may need to restart the kernel after you format
census_2018_formatted["Median household income"] = census_2018_FINAL["Median household income"].map("${:.2f}".format)
census_2018_formatted["Per capita income"] = census_2018_FINAL["Per capita income"].map("${:.2f}".format)
census_2018_formatted["Population"] = census_2018_formatted["Population"].map("{:,.0f}".format)
census_2018_formatted["Poverty count"] = census_2018_formatted["Poverty count"].map("{:,.0f}".format)
census_2018_formatted["Poverty rate"] = census_2018_formatted["Poverty rate"].map("{:.2%}".format)
census_2018_formatted["Unemployment rate"] = census_2018_formatted["Unemployment rate"].map("{:.2%}".format)

census_2018_formatted = census_2018_formatted.reset_index(drop=True)
census_2018_formatted.head()

,Corridor,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Highlandtown,010200,24510010200,"2,869",$107617.00,$56058.00,169,590.00%,100.00%,2100.0,...,107.0,53.0,69.0,107.0,2332.0,1103.0,290.0,340.0,2300.0,229.0
1,Brooklyn,020300,24510020300,"3,877",$108516.00,$87413.00,359,930.00%,140.00%,2912.0,...,89.0,229.0,265.0,364.0,1687.0,793.0,1350.0,243.0,3317.0,317.0
2,E Monument St,040100,24510040100,"4,404",$62500.00,$49876.00,870,1980.00%,150.00%,2863.0,...,894.0,243.0,380.0,913.0,270.0,193.0,2403.0,412.0,3887.0,105.0
3,Hamilton Lauraville,040200,24510040200,901,$34375.00,$16596.00,66,730.00%,190.00%,367.0,...,311.0,55.0,26.0,123.0,0.0,0.0,228.0,18.0,872.0,11.0
4,E Monument St,060200,24510060200,"3,241",$72465.00,$32023.00,864,2670.00%,450.00%,1681.0,...,1370.0,96.0,430.0,58.0,1574.0,534.0,545.0,836.0,2226.0,179.0


In [50]:
census_2018_formatted.dtypes

Corridor                                           object
Census_tract                                       object
GEOID                                               int64
Population                                         object
Median household income                            object
Per capita income                                  object
Poverty count                                      object
Poverty rate                                       object
Unemployment rate                                  object
# employed, age 16+                               float64
Unemployment count                                float64
# persons age 25+ graduated high school           float64
# persons age 25+ with Bachelor's degree          float64
Median age                                        float64
Pop. white                                        float64
Pop. Black                                        float64
Pop. 2 or more races                              float64
Pop. Hispanic 

In [51]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

census_2018_FINAL.to_csv("CommCorr_Census_Stats_2018.csv", index = False, header=True)